In [1]:
import numpy as np
# from ax import ParameterType, RangeParameter
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties

In [2]:
ax_client = AxClient()

ax_client.create_experiment(
    name="moo_experiment1",
    parameters=[
        {
            "name": f"m1-3",
            "type": "range",
            "bounds": [250, 750],
            "value_type": "int"},
        {
            "name": f"m4-6",
            "type": "range",
            "bounds": [25, 100],
            "value_type": "int"},
        {
            "name": f"m7-9",
            "type": "range",
            "bounds": [8, 50],
            "value_type": "int"},
        {
            "name": f"m10-15",
            "type": "range",
            "bounds": [5, 20],
            "value_type": "int"},
        {
            "name": f"depth_scale",
            "type": "range",
            "bounds": [5, 20],
            "value_type": "float"},

    ],
    objectives={
        # `threshold` arguments are optional
        "score": ObjectiveProperties(minimize=False),
        "time_per_move": ObjectiveProperties(minimize=True)
    },
    overwrite_existing_experiment=True,
    # is_test=True,
)

[INFO 07-07 23:51:44] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 07-07 23:51:44] ax.service.utils.instantiation: Due to non-specification, we will use the heuristic for selecting objective thresholds.
[INFO 07-07 23:51:44] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='m1-3', parameter_type=INT, range=[250, 750]), RangeParameter(name='m4-6', parameter_type=INT, range=[25, 100]), RangeParameter(name='m7-9', parameter_type=INT, range=[8, 50]), RangeParameter(name='m10-15', parameter_type=INT, range=[5, 20]), RangeParameter(name='depth_scale', parameter_type=FLOAT, range=[5.0, 20.0])], parameter_constraints=[]).
[INFO 07-07 23:51:44] ax.modelbridge.dispatch_utils: Using Models.MOO since there are more ordered parameters than there are categories for the unordered categorical p

In [3]:
from main import Game
from AI import MDP2
import statistics

def evaluate(parameters):  # figuirng out a real evaluation funciton is all I need


    param_array = np.array([
        parameters["m1-3"],
        parameters["m4-6"],
        parameters["m7-9"],
        parameters["m10-15"],
        parameters["depth_scale"],
    ])
    scores = [0]  # these 0s should be deleted of course
    time_per_moves = [0]
    num_runs = 0
    for game_run in range(num_runs):
        g = Game(use_gui=False)
        g.setup_board()
        m = MDP2(g, game_obj=Game, verbose=False, best_proportion=1, core_params=param_array)
        current_score, current_time_per_move = m.run()
        scores.append(current_score)
        time_per_moves.append(current_time_per_move)
    to_return = {"score": statistics.mean(scores),
                 "time_per_move": statistics.mean(time_per_moves)}

    return {"score": 1, "time_per_move": 2}  # random thing to try so it doesn't actually run and blow up my computer



In [4]:
from tqdm import tqdm
for i in tqdm(range(15)):
    parameters, trial_index = ax_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))

  0%|          | 0/15 [00:00<?, ?it/s][INFO 07-07 23:51:45] ax.service.ax_client: Generated new trial 0 with parameters {'m1-3': 626, 'm4-6': 100, 'm7-9': 47, 'm10-15': 19, 'depth_scale': 11.934898}.
[INFO 07-07 23:51:45] ax.service.ax_client: Completed trial 0 with data: {'score': (1.0, None), 'time_per_move': (2.0, None)}.
[INFO 07-07 23:51:45] ax.service.ax_client: Generated new trial 1 with parameters {'m1-3': 718, 'm4-6': 33, 'm7-9': 38, 'm10-15': 20, 'depth_scale': 13.407911}.
[INFO 07-07 23:51:45] ax.service.ax_client: Completed trial 1 with data: {'score': (1.0, None), 'time_per_move': (2.0, None)}.
[INFO 07-07 23:51:45] ax.service.ax_client: Generated new trial 2 with parameters {'m1-3': 586, 'm4-6': 95, 'm7-9': 34, 'm10-15': 5, 'depth_scale': 6.422668}.
[INFO 07-07 23:51:45] ax.service.ax_client: Completed trial 2 with data: {'score': (1.0, None), 'time_per_move': (2.0, None)}.
[INFO 07-07 23:51:45] ax.service.ax_client: Generated new trial 3 with parameters {'m1-3': 340, 'm4